In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from models.galata import Galata
import pandas as pd

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = Galata(device=device)

In [ ]:
model.load_state_dict(torch.load('galata.pth'), strict=False)

In [ ]:
model.to(device)

In [ ]:
with torch.no_grad():
    model.eval()
    print(model('dataset/57/57.jpg', torch.Tensor([
            [
            0.54,
            0.4,
            0.0,
            0.0,
            0.0,
            0.06,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0
            ]]).to(device)))

In [ ]:
torch.save(model.state_dict(), 'galata.pth')

In [ ]:
df = pd.read_csv('data/csv/fraction.csv')

In [ ]:
df.iloc[9,:]

In [ ]:
a = torch.load('datasets/flexibility_test.pth')

In [ ]:
a[3]

In [ ]:
import clip
from PIL import Image 
import json

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
_,pre = clip.load("ViT-B/32", device='cuda:0')
        

In [ ]:
with open('data/json/flexibility_train.json')as f:
    j = json.load(f)

In [ ]:
image = [pre(Image.open(i['img'])).unsqueeze(0)for i in j]
portion = [i['portion']for i in j]
label = [i['label'] -1 for i in j]

In [ ]:
da = {}
for i in['softness', 'smoothness', 'flexibility', 'thickness']:
    for j in ['train']:
        with open(f'data/json/{i}_{j}.json')as f:
            j = json.load(f)
        j = [{'img':im, 'portion':i['portion'], 'label' :i['label']}for i in j for im in i['img']]
        image = torch.stack([pre(Image.open(i['img']))for i in j])
        portion = torch.Tensor([i['portion']for i in j])
        label = torch.Tensor([i['label'] -1 for i in j])
        print(image.shape)

        torch.save(TensorDataset(image, portion, label), f'datasets/{i}_train.pth')

In [ ]:
for i in['softness', 'smoothness', 'flexibility', 'thickness']:
    for j in ['test']:
        with open(f'data/json/{i}_{j}.json')as f:
            j = json.load(f)
        image = torch.stack([pre(Image.open(i['img']))for i in j])
        portion = torch.Tensor([i['portion']for i in j])
        label = torch.Tensor([i['label'] -1 for i in j])
        print(image.shape)
        torch.save(TensorDataset(image, portion, label), f'datasets/{i}_test.pth')

In [ ]:
[{'img':im, 'portion':i['portion'], 'label' :i['label']}for i in j for im in i['img']]

In [ ]:
torch.stack([torch.load('datasets/dataset_flexibility_train.pth')[0][0], torch.load('datasets/dataset_flexibility_train.pth')[1][0]]).shape

In [ ]:
image.shape